**Scraper for historical odds and results data from Oddsportal.com**

The function scrape_oddsportal(url) in the block below, inputs a url of a page, within the results section of a league in a given year, such as 'https://www.oddsportal.com/baseball/usa/mlb-2024/results/#/page/1/', scrapes all match statistics on that page and outputs the corresponding dataframe.

In [1]:
import os
import time
import threading
import pandas as pd
from math import nan
from multiprocessing.pool import ThreadPool
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from datetime import datetime

# WebDriver wrapper class for managing Selenium driver lifecycle
class Driver:
    def __init__(self):
        options = webdriver.ChromeOptions()
        # Uncomment the following line for headless mode
        # options.add_argument("--headless")
        options.add_experimental_option('excludeSwitches', ['enable-logging'])
        self.driver = webdriver.Chrome(options=options)

    def __del__(self):
        # Ensure driver is properly quit when the object is deleted
        try:
            self.driver.quit()
        except Exception as e:
            print(f"Error quitting driver: {e}")

# Thread-local storage for WebDriver instances
threadLocal = threading.local()

def create_driver():
    """
    Create or retrieve a thread-local WebDriver instance.
    """
    the_driver = getattr(threadLocal, 'the_driver', None)
    if the_driver is None:
        the_driver = Driver()
        setattr(threadLocal, 'the_driver', the_driver)
    return the_driver.driver

# Class to hold scraped game data
class GameData:
    def __init__(self):
        self.date = []
        self.time = []
        self.game = []
        self.score = []
        self.home_odds = []
        self.draw_odds = []
        self.away_odds = []
        self.country = []
        self.league = []

def generate_matches(pgSoup, defaultVal=nan):
    """
    Extract match data from the parsed page source.

    Args:
        pgSoup (BeautifulSoup): Parsed page source.
        defaultVal: Default value for missing data.

    Returns:
        list: List of dictionaries containing match data.
    """
    evtSel = {
        'time': 'p.whitespace-nowrap', #TODO: Fix.
        'game': 'a div:has(>a[title])',
        'score': 'a a[title]+div:has(+a[title])',
        'home_odds': 'a:has(a[title])~div:not(.hidden)',
        'draw_odds': 'a:has(a[title])~div:not(.hidden)+div:nth-last-of-type(3)',
        'away_odds': 'a:has(a[title])~div:nth-last-of-type(2)',
    }

    events = []
    current_group = {}
    pgDate = pgSoup.select_one('h1.title[id="next-matches-h1"]')
    if pgDate:
        pgDate = pgDate.get_text().split(',', 1)[-1].strip()

    for evt in pgSoup.select('div[set]>div:last-child'):
        if evt.parent.select(f':scope>div:first-child+div+div'):
            cgVals = [v.get_text(' ').strip() if v else defaultVal for v in [
                evt.parent.select_one(s) for s in
                [':scope>div:first-child+div>div:first-child',
                 ':scope>div:first-child>a:nth-of-type(2):nth-last-of-type(2)',
                 ':scope>div:first-child>a:nth-of-type(3):last-of-type']]]
            current_group = dict(zip(['date', 'country', 'league'], cgVals))
            if pgDate:
                current_group['date'] = pgDate

        evtRow = {'date': current_group.get('date', defaultVal)}

        for k, v in evtSel.items():
            element = evt.select_one(v)
            text = element.get_text(' ') if element else defaultVal
            evtRow[k] = ' '.join(text.split()) if isinstance(text, str) else text

        evtTeams = evt.select('a div>a[title]')
        evtRow['game'] = ' – '.join(a['title'] for a in evtTeams)
        evtRow['country'] = current_group.get('country', defaultVal)
        evtRow['league'] = current_group.get('league', defaultVal)

        events.append(evtRow)
    return events

def parse_data(url):
    """
    Parse match data from the given URL.

    Args:
        url (str): URL to scrape.

    Returns:
        GameData: Scraped match data.
    """
    browser = create_driver()
    try:
        browser.get(url)

        # Wait for elements to load
        WebDriverWait(browser, 10).until(EC.presence_of_all_elements_located(
            (By.CSS_SELECTOR, "div[set]>div:last-child a:has(a[title])~div:not(.hidden)")))

        # Scroll through the page to load all content
        scroll_pause_time = 5
        last_height = browser.execute_script("return document.body.scrollHeight")
        time.sleep(5)
        while True:
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight/2);")
            time.sleep(scroll_pause_time)
            new_height = browser.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Parse page source
        soup = bs(browser.page_source, "lxml")
        game_data = GameData()
        game_keys = [key for key, value in game_data.__dict__.items() if isinstance(value, list)]
        for row in generate_matches(soup, defaultVal=nan):
            for k in game_keys:
                getattr(game_data, k).append(row.get(k, nan))

        return game_data
    except Exception as e:
        print(f"Error parsing data from {url}: {e}")
        return None

def scrape_oddsportal(url):
    """
    Scrape match data from OddsPortal and return as a DataFrame.

    Args:
        url (str): URL of the OddsPortal page to scrape.

    Returns:
        pd.DataFrame: DataFrame containing scraped match data.
    """
    pool = ThreadPool(5)  # Use thread pool for parallel scraping
    game_data_results = pool.imap(parse_data, [url])

    game_data_df_list = []
    for game_data in game_data_results:
        if game_data is not None:
            try:
                game_data_df_list.append(pd.DataFrame(game_data.__dict__))
            except Exception as e:
                print(f'Error tabulating game data: {e}')

    try:
        games = pd.concat(game_data_df_list, ignore_index=True)
    except Exception as e:
        print(f'Error concatenating DataFrames: {e}')
        games = None

    # Garbage collection (freeing unused memory). Could be commented.
    import gc
    gc.collect()

    return games

The function below saves excel sheet of all tournament results of a given league and season start year to a folder like "MLB_Data" in cwd. If process is interrupted, rerun and it will pick up where it left off. To restart the process for another year, remove the previous output file from the designated folder.

In [2]:
def generate_and_save_tournament_results(league, year, number_of_pages):
    """
    Fetches match data for a specified league and season, aggregating results from multiple pages,
    and saves the data to an Excel file. The function is designed to handle interruptions gracefully,
    allowing it to resume from where it left off.

    Args:
        league (str): The league identifier (e.g., 'MLB', 'NBA', 'NFL', 'NHL').
        year (int): The starting year of the season.
        number_of_pages (int): The total number of pages to fetch data from.

    Returns:
        None
    """
    # Define league-specific URL prefixes and data directories
    league_info = {
        'MLB': {
            'url_prefix': f'https://www.oddsportal.com/baseball/usa/mlb-{year}/results/#/page/',
            'data_folder': 'MLB_Data'
        },
        'NBA': {
            'url_prefix': f'https://www.oddsportal.com/basketball/usa/nba-{year}-{year + 1}/results/#/page/',
            'data_folder': 'NBA_Data'
        },
        'NFL': {
            'url_prefix': f'https://www.oddsportal.com/american-football/usa/nfl-{year}-{year + 1}/results/#/page/',
            'data_folder': 'NFL_Data'
        },
        'NHL': {
            'url_prefix': f'https://www.oddsportal.com/hockey/usa/nhl-{year}-{year + 1}/results/#/page/',
            'data_folder': 'NHL_Data'
        }
    }

    # Validate league input
    if league not in league_info:
        raise ValueError(f"Unsupported league '{league}'. Supported leagues are: {', '.join(league_info.keys())}.")

    # Extract URL prefix and data folder for the specified league
    url_prefix = league_info[league]['url_prefix']
    data_folder = league_info[league]['data_folder']

    # Ensure the data directory exists
    os.makedirs(data_folder, exist_ok=True)

    # Define the path for the Excel file
    excel_file_path = os.path.join(data_folder, f"{league}_{year}_results.xlsx")

    # Initialise or load existing data
    if os.path.exists(excel_file_path):
        all_games = pd.read_excel(excel_file_path, index_col=0)
        start_page = (len(all_games) // 50) + 1
    else:
        all_games = pd.DataFrame()
        start_page = 1

    # Fetch data from each page
    for page in range(start_page, number_of_pages + 1):
        url = f"{url_prefix}{page}/"
        try:
            page_games_df = scrape_oddsportal(url)
            
            # It happens quite often that 45 out of 50 matches on a page are loaded.
            # So we simply halt if we have 45 match statistics on a given page.
            # In very rare cases of there being exactly 45 matches on the last page, this could skip a healthy page.
            # TODO: Fix this.
            if len(page_games_df)==45:
                break
                print(f"Page {page} yielded 45 results and likely missed 5.")            
            if page_games_df is not None and not page_games_df.empty:
                all_games = pd.concat([all_games, page_games_df], ignore_index=True)
                # Save progress after each successful page fetch
                all_games.to_excel(excel_file_path)
                print(f"Page {page} data fetched and saved successfully.")
            else:
                print(f"No data found on page {page}.")
                break
        except Exception as e:
            print(f"Error fetching data from page {page}: {e}")
            # Log the error with a timestamp
            with open(os.path.join(data_folder, 'error_log.txt'), 'a') as log_file:
                log_file.write(f"{datetime.now()}: Error on page {page} - {e}\n")
            # Optionally, implement a retry mechanism or continue to the next page
            break
#            continue
    if page == number_of_pages:
        print(f"Data collection completed. Results saved to {excel_file_path}.")
    else:
        print(f"Data collection interrupted. Results, collected thus far, saved to {excel_file_path}.")


Testing for 2024 season of Major League Baseball, which contains 50 pages of match results.

In [3]:
generate_and_save_tournament_results('MLB', 2024, 50)

Page 1 data fetched and saved successfully.
Page 2 data fetched and saved successfully.
Page 3 data fetched and saved successfully.
Page 4 data fetched and saved successfully.
Page 5 data fetched and saved successfully.
Page 6 data fetched and saved successfully.
Page 7 data fetched and saved successfully.
Page 8 data fetched and saved successfully.
Page 9 data fetched and saved successfully.
Page 10 data fetched and saved successfully.
Page 11 data fetched and saved successfully.
Page 12 data fetched and saved successfully.
Page 13 data fetched and saved successfully.
Page 14 data fetched and saved successfully.
Page 15 data fetched and saved successfully.
Page 16 data fetched and saved successfully.
Page 17 data fetched and saved successfully.
Page 18 data fetched and saved successfully.
Page 19 data fetched and saved successfully.
Page 20 data fetched and saved successfully.
Page 21 data fetched and saved successfully.
Page 22 data fetched and saved successfully.
Page 23 data fetche

Validating the data.

In [5]:
# loading the generated excel file into a dataframe.
mlb_2024_df = pd.read_excel(os.path.join(os.getcwd(), "MLB_Data\MLB_2024_results.xlsx"))

In [6]:
print(mlb_2024_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2428 entries, 0 to 2427
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  2428 non-null   int64  
 1   date        2428 non-null   object 
 2   time        0 non-null      float64
 3   game        2428 non-null   object 
 4   score       2428 non-null   object 
 5   home_odds   2428 non-null   object 
 6   draw_odds   0 non-null      float64
 7   away_odds   2428 non-null   object 
 8   country     2428 non-null   object 
 9   league      2428 non-null   object 
dtypes: float64(2), int64(1), object(7)
memory usage: 189.8+ KB
None


The following reformats excel files in relevant directory (here MLB_Data), and saves them in a subfolder "Fixed". (E.g. breaking scores like '1-3' into home_score and away_score columns, etc.

In [9]:
# Define directories
current_working_directory = os.getcwd()
raw_data_directory = os.path.join(current_working_directory, "MLB_Data")
fixed_data_directory = os.path.join(current_working_directory, "MLB_Data", "Fixed")

# Ensure the output directory exists
os.makedirs(fixed_data_directory, exist_ok=True)

# List all files in the raw data directory
excel_files = os.listdir(raw_data_directory)

# Process each file in the directory
for file_index, file_name in enumerate(excel_files):
    raw_file_path = os.path.join(raw_data_directory, file_name)  # Full path to input file
    fixed_file_path = os.path.join(fixed_data_directory, file_name)  # Full path to output file

    if os.path.isdir(raw_file_path):
        # Skip directories if accidentally included in the list
        continue

    # Read the current Excel file
    try:
        raw_data_table = pd.read_excel(raw_file_path)
    except Exception as e:
        print(f"Error reading file {file_name}: {e}")
        continue

    # Initialise a DataFrame for reformatted data
    reformatted_data = pd.DataFrame(columns=[
        'date', 'match_type', 'home_team', 'away_team', 'result',
        'home_score', 'away_score', 'home_odds', 'away_odds'
    ])

    # Process each row in the raw data
    for row_index in range(len(raw_data_table)):
        # Extract and format the date
        raw_date = raw_data_table['date'][row_index]
        if '-' in raw_date:
            raw_date = raw_date.split('  -')[0]
        formatted_date = pd.to_datetime(raw_date, format='mixed', dayfirst=True).date()

        # Determine the match type
        if 'Play Off' in raw_data_table['date'][row_index]:
            match_type = 'play_off'
        elif 'Pre-season' in raw_data_table['date'][row_index]:
            match_type = 'pre_season'
        else:
            match_type = 'regular'

        # Extract home and away teams
        home_team, away_team = raw_data_table['game'][row_index].split(' – ')

        # Extract scores, handling missing or invalid values
        try:
            home_score = int(raw_data_table['score'][row_index].split(' – ')[0])
            away_score = int(raw_data_table['score'][row_index].split(' – ')[1])
        except (ValueError, AttributeError):
            home_score = ''
            away_score = ''

        # Determine match result
        if isinstance(home_score, int) and isinstance(away_score, int):
            result = 'H' if home_score > away_score else 'A'
        else:
            result = ''

        # Extract odds, handling missing or invalid values
        try:
            home_odds = float(raw_data_table['home_odds'][row_index])
            away_odds = float(raw_data_table['away_odds'][row_index])
        except (ValueError, TypeError):
            home_odds = ''
            away_odds = ''

        # Append the processed row to the reformatted DataFrame
        reformatted_data.loc[len(reformatted_data)] = [
            formatted_date, match_type, home_team, away_team, result,
            home_score, away_score, home_odds, away_odds
        ]

    # Reset the index of the reformatted DataFrame
    reformatted_data.reset_index(drop=True, inplace=True)

    # Save the reformatted data to the fixed data directory
    try:
        reformatted_data.to_excel(fixed_file_path, index=False)
        print(f"Processed and saved: {file_name}")
    except PermissionError as e:
        print(f"Permission error saving file {file_name}: {e}")
    except Exception as e:
        print(f"Unexpected error saving file {file_name}: {e}")

Processed and saved: MLB_2024_results.xlsx


In [10]:
# loading the formatted excel file into a dataframe.
mlb_2024_df_formatted = pd.read_excel(os.path.join(os.getcwd(), "MLB_Data\Fixed\MLB_2024_results.xlsx"))

In [11]:
print(mlb_2024_df_formatted.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2428 entries, 0 to 2427
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        2428 non-null   datetime64[ns]
 1   match_type  2428 non-null   object        
 2   home_team   2428 non-null   object        
 3   away_team   2428 non-null   object        
 4   result      2427 non-null   object        
 5   home_score  2427 non-null   float64       
 6   away_score  2427 non-null   float64       
 7   home_odds   2424 non-null   float64       
 8   away_odds   2424 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(4)
memory usage: 170.8+ KB
None


In [12]:
mlb_2024_df_formatted.head(10)

,date,match_type,home_team,away_team,result,home_score,away_score,home_odds,away_odds
0,2024-10-31,play_off,New York Yankees,Los Angeles Dodgers,A,6.0,7.0,1.67,2.27
1,2024-10-31,play_off,New York Yankees,Los Angeles Dodgers,H,11.0,4.0,1.74,2.15
2,2024-10-31,play_off,New York Yankees,Los Angeles Dodgers,A,2.0,4.0,1.67,2.27
3,2024-10-31,play_off,Los Angeles Dodgers,New York Yankees,H,4.0,2.0,1.75,2.14
4,2024-10-31,play_off,Los Angeles Dodgers,New York Yankees,H,6.0,3.0,1.82,2.05
5,2024-10-31,play_off,Los Angeles Dodgers,New York Mets,H,10.0,5.0,1.66,2.30
6,2024-10-31,play_off,Cleveland Guardians,New York Yankees,A,2.0,5.0,2.13,1.76
7,2024-10-31,play_off,Cleveland Guardians,New York Yankees,A,6.0,8.0,2.05,1.81
8,2024-10-31,play_off,New York Mets,Los Angeles Dodgers,H,12.0,6.0,2.13,1.76
9,2024-10-31,play_off,New York Mets,Los Angeles Dodgers,A,2.0,10.0,2.15,1.75


The following code could be used to concatenate excel files in a folder and save as a single excel file.

In [ ]:
# Set up the directory
cwd = os.getcwd()
directory = os.path.join(cwd, "MLB_Data", "Fixed")

# List all files in the directory
files = os.listdir(directory)

# Filter for Excel files (e.g., .xlsx and .xls)
excel_files = [file for file in files if file.endswith(('.xlsx', '.xls'))]

# Initialise an empty list to hold DataFrames
dataframes = []

# Read each Excel file and append to the list
for file in excel_files:
    file_path = os.path.join(directory, file)
    try:
        df = pd.read_excel(file_path)
        dataframes.append(df)
    except Exception as e:
        print(f"Error reading {file}: {e}")

# Concatenate all DataFrames
if dataframes:
    combined_df = pd.concat(dataframes, ignore_index=True)
    combined_df.sort_values(by="date", ascending=False, inplace=True)

    # Save the resulting DataFrame to a new Excel file
    output_file = os.path.join(directory, "combined_data.xlsx")
    combined_df.to_excel(output_file, index=False)
    print(f"Combined Excel file saved as: {output_file}")
else:
    print("No Excel files found to concatenate.")